In [1]:
import numpy as np

In [241]:
#Single qubit states
v0 = np.array([[1],[0]])
v1 = np.array([[0],[1]])

#One qubit gates
H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
T = np.array([[1,0],[0, np.exp(1j*np.pi/4)]])

#Acting one qubit gates
def H_op(v):
    return H @ v

def X_op(v):
    return X @ v
    
def Y_op(v):
    return Y @ v
    
def Z_op(v):
    return Z @ v

def T_op(v):
    return T @ v

def phase_arb_op(v,theta):
    P = np.array([[1,0],[0, np.exp(1j*theta/2)]])
    return P @ v

In [279]:
#Two qubit gates
CNOT = np.kron(np.outer(v0,v0), np.identity(2)) + np.kron(np.outer(v1,v1), X)
rCNOT = np.kron(np.identity(2), np.outer(v0,v0)) + np.kron(X, np.outer(v1,v1))
SWAP = CNOT @ rCNOT @ CNOT

#Acting two qubit gates
def CNOT_op(v, w, r = False):
    if r == True:
        return rCNOT @ np.kron(v, w)
    else:
        return CNOT @ np.kron(v, w)

In [405]:
#Setting up the Bell states
def Bell(x = 0, z = 0):
    if (x == 0 and z == 0):
        psi_0 = np.kron(v0,v0)
    if (x == 1 and z == 0):
        psi_0 = np.kron((v0),X_op(v0))
    if (x == 0 and z == 1):
        psi_0 = np.kron(X_op(v0),(v0))
    if (x == 1 and z == 1):
        psi_0 = np.kron(X_op(v0),X_op(v0))
    psi_1 = np.kron(np.identity(2),H) @ psi_0
    psi_2 = rCNOT @ psi_1
    return psi_2

In [469]:
np.log2(8)

3.0

In [693]:
#Setting up measurements
from functools import reduce
def kron_all(matrices, n):
    """
    Compute the Kronecker product of a list of matrices.
    Equivalent to applying np.kron multiple times.
    """
    if n == 0:
        return 1
    else:
        return reduce(np.kron, [matrices]*n)


rng = np.random.default_rng()

#Single qubit measurement
def measurement_single(psi):
    output = rng.choice((0,1), p=[np.tensordot(psi,v0)**2, np.tensordot(psi,v1)**2])
    if output:
        state = v1
    else:
        state = v0
    return output, state

def measurement_multiple(psi, m = 1):
    q = int(np.log2(len(psi)))
    projector_0 = np.kron(kron_all(np.identity(2),q-m),np.kron(np.outer(v0,v0),kron_all(np.identity(2),m-1)))
    projector_1 = np.kron(kron_all(np.identity(2),q-m),np.kron(np.outer(v1,v1),kron_all(np.identity(2),m-1)))
    state_0 = projector_0 @ psi
    state_1 = projector_1 @ psi
    print(np.tensordot(psi,state_0)**2, np.tensordot(psi,state_1)**2)
    print(state_0, state_1)
    print(projector_0, projector_1)
    #return output, state

In [ ]:
    output = rng.choice((0,1), p=[np.tensordot(psi,state_0)**2, np.tensordot(psi,state_1)**2])
    if output:
        state = state_0
    else:
        state = state_1

In [671]:
q = int(np.log2(len(psi_1)))
q-1-1

1

In [673]:
measurement_multiple(psi_1, m = 2)

3 1 1


(array([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]]))

In [579]:
measurement_single(H_op(v0))

(1,
 array([[0],
        [1]]))

q = int(np.log2(len(psi_1)))
n = 2
(np.kron(np.kron(np.identity(2**(q-n)),v0),np.identity(2**(n-1))), psi_1)

In [695]:
#Teleportation scheme
tele_qubit = v0
psi_0 = np.kron(Bell(0,0),tele_qubit)
psi_1 = np.kron(np.identity(2),rCNOT) @ psi_0
measurement_multiple(psi_1, 2)

0.2499999999999999 0.2499999999999999
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]] [[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]
 [0.        ]]
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]] [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [239]:
SWAP @ np.kron(v0,v1), np.kron(v0,v1)

(array([[0.],
        [0.],
        [1.],
        [0.]]),
 array([[0],
        [1],
        [0],
        [0]]))

In [109]:
v1 = np.array([[1],[0]])
v2 = 1/np.sqrt(2) * np.array([[1], [-1]])
psi = X(T(hadamard(v)))
psi

array([[ 7.07106781e-01+0.70710678j],
       [-2.23711432e-17+0.j        ]])

In [57]:
H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])
v = 1/np.sqrt(2) * np.array([[1], [-1]])
H @ v

array([[-2.23711432e-17],
       [ 1.00000000e+00]])